# Armazéns Hanoi Simulation: Experimental Results Analysis

This notebook loads, aggregates, and visualizes the results of all simulation experiments. It provides code and visualizations for interpreting the computational and operational performance of the system across all experimental scenarios.

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import json
import os

# Set up matplotlib for inline plotting
%matplotlib inline

## Load and Aggregate Experimental Results

This section loads all `metrics.json` files from the results directory, aggregates them into a single DataFrame, and extracts experiment parameters for analysis.

In [ ]:
# Load all metrics.json files and aggregate results
results_dir = os.path.join('..', 'results')
metrics_files = glob.glob(os.path.join(results_dir, '*', 'metrics.json'))
meta_files = [f.replace('metrics.json', os.path.basename(f).replace('metrics.json', 'in.txt.meta.json')) for f in metrics_files]

records = []
for mfile, metafile in zip(metrics_files, meta_files):
    with open(mfile) as f:
        metrics = json.load(f)
    params = {}
    if os.path.exists(metafile):
        with open(metafile) as mf:
            meta = json.load(mf)
            params = meta.get('params', {})
    record = {**params, **metrics, 'metrics_file': mfile}
    records.append(record)

df = pd.DataFrame(records)
display(df.head())
print(f"Loaded {len(df)} experiment results.")

## Visualize Key Metrics

This section provides visualizations for the main metrics across all experiments:
- Execution time, memory usage, heap/stack operations
- Package delivery times (mean, median, 95th percentile, max)
- Re-storage events, warehouse section depth, transport utilization
- Line plots, histograms, and heatmaps as described in the experimental plan

In [ ]:
# Example: Execution Time vs. Number of Warehouses (Experiment A)
expA = df[df['metrics_file'].str.contains('expA_')]
plt.figure(figsize=(8,5))
plt.plot(expA['numeroarmazens'], expA['execution_time'], marker='o')
plt.xlabel('Number of Warehouses')
plt.ylabel('Total Execution Time (s)')
plt.title('Experiment A: Execution Time vs. Number of Warehouses')
plt.grid(True)
plt.show()

In [ ]:
# Example: Histogram of Package Delivery Times (select an experiment)
example = df.iloc[0]  # Change index to select different experiment
if 'delivery_time_stats' in example and isinstance(example['delivery_time_stats'], dict):
    stats = example['delivery_time_stats']
    # If you have raw delivery times, plot histogram
    # Otherwise, plot summary stats
    plt.figure(figsize=(6,4))
    plt.bar(['mean', 'median', 'p95', 'max'], [stats.get(k, 0) for k in ['mean','median','p95','max']])
    plt.title('Package Delivery Time Stats')
    plt.ylabel('Time')
    plt.show()
else:
    print('No delivery time stats available in this result.')

In [ ]:
# Example: Heatmap for Experiment C (capacity vs. interval, color = re-storage events)
expC = df[df['metrics_file'].str.contains('expC_')]
heatmap_data = expC.pivot_table(index='capacidadetransporte', columns='intervalotransportes', values='re_storage_events', aggfunc='mean')
plt.figure(figsize=(8,6))
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='YlOrRd')
plt.title('Experiment C: Re-storage Events Heatmap')
plt.xlabel('Transport Interval')
plt.ylabel('Transport Capacity')
plt.show()

## Interpretation and Summary of Findings

Use the visualizations above to interpret the results for each experiment:
- Identify trends, bottlenecks, and inflection points
- Relate observed behaviors to the underlying data structures and algorithms
- Summarize key findings and recommendations for scaling the Armazéns Hanoi system

*Fill in your analysis and conclusions below.*